# About: Prerequisites for Literate Computing via Notebooks
文芸的機械化の準備 - Notebookでね

----

This notebook describes enviromental prerequisits in order to utilize the set of notebooks under this directory.
Prerequisits cover both for the originate host where notebooks operate and for the target hosts manipulated by notebooks.

ここでは本ディレクトリに含まれるNotebookを利用するために必要となる環境条件を明示的に定義している。
環境条件には、Notebookを動作させているホストマシンの環境、およびNotebookで操作しようとしている対象ホスト群の環境が含まれる。

----

自身のNotebook環境にNotebookをコピーした後、読み進めることで条件を満たしているか確認できる。

条件には推奨の度合いによりレベルがあり、 **A=必須、B=推奨、C=任意** というレベルを設定してある。

このNotebookを確認することで、Notebookで操作しようとしている捜査対象ホスト群が **Seedingが済んだ状態である** ことを確認できる。

# Ansibleの設定

Notebook環境は、以下の条件を満たしている必要がある。

## Ansibleがインストールされていること - A

以下のコマンド実行がエラーとならず、インストールされているAnsibleのバージョンが表示されること。

In [1]:
!ansible --version

ansible XXX.XXX.XXX.0
  config file = /etc/ansible/ansible.cfg
  configured module search path = Default w/o overrides


ansibleコマンドがインストールされていない場合は、[Installing the Control Machine](http://docs.ansible.com/ansible/intro_installation.html#installing-the-control-machine)を参考に、Ansibleをインストールする。

## AnsibleでDefault Moduleが利用可能であること - A

DefaultのModuleが使えればよい。特にカスタマイズすべき項目はなし。*...将来は腐っているモジュールは差し替えたい。*

# Ansible - Binding対象間の設定

このNotebook環境にインストールされているAnsibleは、Binding対象となるホストに対して通信、操作可能な状態になっている必要がある。

## Inventoryに指定されたホストに到達可能なこと - A

AnsibleのInventoryに指定された全ホストに対して、この環境からアクセス可能であること。

- 以下のコマンド実行がエラーもしくは警告(`provided hosts list is empty`)とならないこと
- Inventoryに登録されているホストに関して、すべてSUCCESSとなること

In [5]:
#elasticsearchをインストールするサーバーでansibleコマンドを実行するユーザー
USER='ansible'

#公開鍵認証を行う場合の秘密鍵のパス
KEYPATH='~/.ssh/ansible_id_rsa'

In [21]:
!ansible all -m ping -i /etc/ansible/hosts -u $USER --private-key=$KEYPATH

XXX.XXX.XXX.233 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.234 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.232 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.235 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.236 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


このNotebook環境のAnsibleから、ホストへの認証方法については特に問わない。

> 我々(NIIクラウド運用チーム)の環境ではkeypairによる認証を使ってAnsibleから各ホストにアクセスをしている。
> そのため、以下のpublic keyをBinding対象ホストの `~/.ssh/authorized_keys` に設定している。

## 適切なユーザで認証されること - B

Ansibleからホストへの認証は、一般ユーザとして認証されるようにしておく。

> 我々(NIIクラウド運用チーム)の環境では、ユーザ ansible を作って対応している。

In [23]:
!ansible -a 'whoami' all -i /etc/ansible/hosts -u $USER --private-key=$KEYPATH

XXX.XXX.XXX.232 | SUCCESS | rc=0 >>
ansible

XXX.XXX.XXX.234 | SUCCESS | rc=0 >>
ansible

XXX.XXX.XXX.233 | SUCCESS | rc=0 >>
ansible

XXX.XXX.XXX.236 | SUCCESS | rc=0 >>
ansible

XXX.XXX.XXX.235 | SUCCESS | rc=0 >>
ansible



## Binding対象ホストにてsudo可能であること - A

Inventoryに記述された全ホストに関してbecome, become-userが可能であることを確認しておく。

- 以下のコマンド実行がエラーとならないこと
- Inventoryに登録されているホストに関して、すべてSUCCESSとなり、結果がrootとなること

In [24]:
!ansible --become --become-user root -a 'whoami' all -i /etc/ansible/hosts -u $USER --private-key=$KEYPATH

XXX.XXX.XXX.234 | SUCCESS | rc=0 >>
root

XXX.XXX.XXX.233 | SUCCESS | rc=0 >>
root

XXX.XXX.XXX.232 | SUCCESS | rc=0 >>
root

XXX.XXX.XXX.236 | SUCCESS | rc=0 >>
root

XXX.XXX.XXX.235 | SUCCESS | rc=0 >>
root



以上の項目が満たされていればOK。

# Binding対象の設定

Binding対象に関して、Notebook + Ansibleによる管理を適用する上で適切な設定になっていることを確認する。

## Ansibleによる操作ログが記録されていること - B

NotebookからBinding対象の操作をおこなった際に、過去おこなった操作を適切に参照できる必要がある。

> 我々の環境では、ansibleによる操作は `/var/log/secure` (CentOSの場合) `/var/log/auth.log` (Ubuntuの場合) に記録されている

In [26]:
!ansible -b -m shell -a 'grep ansible- /var/log/* | tail' all -i /etc/ansible/hosts -u $USER --private-key=$KEYPATH

XXX.XXX.XXX.232 | SUCCESS | rc=0 >>
/var/log/secure-20161218:Dec 16 18:37:03 cnXXXX0601 sudo: ansible : TTY=pts/0 ; PWD=/home/ansible ; USER=root ; COMMAND=/bin/sh -c echo BECOME-SUCCESS-bwztadatoytjhnwilwliaramwawsccwl; /usr/bin/python /home/ansible/.ansible/tmp/ansible-tmp-1481881023.13-255647589199629/yum.py; rm -rf "/home/ansible/.ansible/tmp/ansible-tmp-1481881023.13-255647589199629/" > /dev/null 2>&1
/var/log/secure-20161218:Dec 16 18:37:04 cnXXXX0601 sudo: ansible : TTY=pts/0 ; PWD=/home/ansible ; USER=root ; COMMAND=/bin/sh -c echo BECOME-SUCCESS-vpysrxwscjihfbkaudlyrkejuqvctyhf; /usr/bin/python /home/ansible/.ansible/tmp/ansible-tmp-1481881024.57-228453908342457/lineinfile.py; rm -rf "/home/ansible/.ansible/tmp/ansible-tmp-1481881024.57-228453908342457/" > /dev/null 2>&1
/var/log/secure-20161218:Dec 16 18:37:05 cnXXXX0601 sudo: ansible : TTY=pts/0 ; PWD=/home/ansible ; USER=root ; COMMAND=/bin/sh -c echo BECOME-SUCCESS-nrambptrgnjbhwwgbkvuqxyqinguxfry; /usr/bin/python /home/an

### ログファイルが適切にrotateされていること - B

操作の記録がディスクを圧迫しないことを確認しておく。ログは短期的な(Ansibleによる)操作の記録として参照可能にしておくべきものという前提。長期的には、Notebookを証跡としてアーカイブしておくべき。

## 共通の方法でホストの状況が確認できること - C

Notebookによる操作実施時は、適当なチェックポイントごとに状況を記録することで、後戻りしたくなった時に戻れる(戻ったことが確認できる)ようにする。

### OSのバージョン確認

lsb_releaseを使う。

In [27]:
!ansible -a 'lsb_release -a' all -i /etc/ansible/hosts -u $USER --private-key=$KEYPATH

XXX.XXX.XXX.233 | SUCCESS | rc=0 >>
LSB Version:	:core-4.1-amd64:core-4.1-noarch:cxx-4.1-amd64:cxx-4.1-noarch:desktop-4.1-amd64:desktop-4.1-noarch:languages-4.1-amd64:languages-4.1-noarch:printing-4.1-amd64:printing-4.1-noarch
Distributor ID:	CentOS
Description:	CentOS Linux release 7.2.1511 (Core) 
Release:	7.2.1511
Codename:	Core

XXX.XXX.XXX.234 | SUCCESS | rc=0 >>
LSB Version:	:core-4.1-amd64:core-4.1-noarch:cxx-4.1-amd64:cxx-4.1-noarch:desktop-4.1-amd64:desktop-4.1-noarch:languages-4.1-amd64:languages-4.1-noarch:printing-4.1-amd64:printing-4.1-noarch
Distributor ID:	CentOS
Description:	CentOS Linux release 7.2.1511 (Core) 
Release:	7.2.1511
Codename:	Core

XXX.XXX.XXX.232 | SUCCESS | rc=0 >>
LSB Version:	:core-4.1-amd64:core-4.1-noarch:cxx-4.1-amd64:cxx-4.1-noarch:desktop-4.1-amd64:desktop-4.1-noarch:languages-4.1-amd64:languages-4.1-noarch:printing-4.1-amd64:printing-4.1-noarch
Distributor ID:	CentOS
Description:	CentOS Linux release 7.3.1611 (Core) 
Release:	7.3.1611
Codename:	C

## リソースの使用状況変化をモニタリングしていること - C

Binding対象を操作した際に、予期せぬディスク使用量増加やサービス異常終了に気づけるよう、状況変化をモニタリングしていることが望ましい。

## カーネルパラメータ設定

vm.max_map_countの値を"262144"に設定する。

詳細はElasticsearch Referenceの[Virtual memory](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/vm-max-map-count.html)を参照してください。

In [13]:
!ansible-playbook -i /etc/ansible/hosts playbooks/set_kernelparameter.yml -u $USER --private-key=$KEYPATH


PLAY [all] *********************************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.231]

TASK [Set vm.max_map_count to 262144 in /etc/sysctl.conf] **********************
changed: [XXX.XXX.XXX.231]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.231              : ok=2    changed=1    unreachable=0    failed=0   



## 使用portの通信許可設定

Elasticsearchで使用する、9200,9300ポートの通信を許可する。  
通信を許可するIPアドレスは、ご自身の環境に合わせてplaybooksディレクトリにある、([set_iptables-rule.yml](playbooks/set_iptables-rule.yml))を編集してください。

In [29]:
!ansible-playbook -i /etc/ansible/hosts playbooks/set_iptables-rule.yml -u $USER --private-key=$KEYPATH


PLAY [all] *********************************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.234]
ok: [XXX.XXX.XXX.232]
ok: [XXX.XXX.XXX.233]
ok: [XXX.XXX.XXX.235]
ok: [XXX.XXX.XXX.236]

TASK [iptables] ****************************************************************
ok: [XXX.XXX.XXX.232]
ok: [XXX.XXX.XXX.234]
ok: [XXX.XXX.XXX.233]
ok: [XXX.XXX.XXX.236]
ok: [XXX.XXX.XXX.235]

TASK [iptables] ****************************************************************
ok: [XXX.XXX.XXX.232]
ok: [XXX.XXX.XXX.234]
ok: [XXX.XXX.XXX.233]
ok: [XXX.XXX.XXX.235]
ok: [XXX.XXX.XXX.236]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.232              : ok=3    changed=0    unreachable=0    failed=0   
XXX.XXX.XXX.233              : ok=3    changed=0    unreachable=0    failed=0   
XXX.XXX.XXX.234              : ok=3    changed=0    unreachable=0    failed=0   
XXX.XXX.XXX.235  